In [97]:
# imports
from bs4 import BeautifulSoup
from datetime import date
from os.path import exists
import requests, shutil, pandas as pd

In [98]:
# url, request and response
url = 'https://store.steampowered.com/hwsurvey/videocard/'
response = requests.get(url)

In [99]:
soup = BeautifulSoup(response.content)


In [100]:
data = soup.findAll('div', id='sub_stats')[0]

In [101]:
#print(data)

In [102]:
# extracting recent 5 months
data1 = data.findAll('div', class_='col_header')[1:6]
months = []
for month in range(len(data1)):
    months.append(data1[month].text)

In [103]:
# data extraction line for gpus
data2 = data.findAll('div', class_='substats_col_left')[8:109]

In [104]:
# data extraction lines for Recent month column %
data3 = data.findAll('div', class_='substats_col_month_last_pct')[8:109]

In [105]:
# data extraction for other months
data4 = data.findAll('div', class_='substats_col_month')[32:]

In [106]:
# creating months-percent dict
numerate = 0
months_percent = {
    months[0] : [], 
    months[1] : [],
    months[2] : [],
    months[3] : [],
    months[4] : []
}
for percent in data4:

    if (str(months[0]) in percent.text):
        break
    months_percent[months[numerate]].append(percent.text[:-1])
    numerate += 1
    if (numerate == 4):
        numerate = 0

for percent in data3:
    months_percent[months[4]].append(percent.text[:-1])

In [107]:
# Getting list of gpus
gpu_percent_dict = {}
num = 0
for entry in data2:
    temp = [
        months_percent[months[0]][num],
        months_percent[months[1]][num],
        months_percent[months[2]][num],
        months_percent[months[3]][num],
        months_percent[months[4]][num]
    ]
    if ( ('NVIDIA' in entry.text) or ('AMD' in entry.text) or ('Intel' in entry.text) ):
        gpu_percent_dict[entry.text.strip()] = temp
    if ( ('Other' in entry.text)):
        gpu_percent_dict['Other'] = temp

    num += 1

In [108]:
#print(gpu_percent_dict)

In [109]:
# create dataframe
df = pd.DataFrame.from_dict(gpu_percent_dict, orient='index', columns=[
    months[0],
    months[1],
    months[2],
    months[3],
    months[4]])

In [110]:
# Dataframe Reorganization
df['GPU'] = df.index
df = df[['GPU',months[0],months[1],months[2],months[3],months[4]]]
df.reset_index(drop=True)

,GPU,JUL,AUG,SEP,OCT,NOV
0,NVIDIA GeForce GTX 1060,9.17,10.46,8.64,8.08,7.83
1,NVIDIA GeForce GTX 1650,5.62,5.36,5.99,5.94,6.10
2,NVIDIA GeForce GTX 1050 Ti,6.41,6.14,6.16,5.94,5.92
3,NVIDIA GeForce RTX 2060,5.52,6.87,5.57,5.16,5.24
4,NVIDIA GeForce GTX 1050,3.74,3.35,3.43,3.39,3.30
...,...,...,...,...,...,...
96,AMD Radeon RX Vega,0.19,0.16,0.17,0.17,0.16
97,NVIDIA GeForce RTX 2060 with Max-Q Design,,,0.15,,0.16
98,NVIDIA GeForce GTX 550 Ti,0.15,,,,0.15
99,Intel Valleyview Baytrail,0.18,0.15,0.15,0.15,0.15


In [111]:
# test df methods
len(df)

101

In [112]:
# writing to csv with current month of data collection

file_name = f'steam_gpu_survey-{date.today().month}-{date.today().year}'

# check if file already exists
if (~exists("/data/{file_name}")):

    # writing csv file
    df.to_csv(path_or_buf=file_name, index=False)

    # moving csv file to data directory
    source = f'{file_name}'
    destination = "data"
    shutil.move(source, destination)
else:
    print("File Already Exists")